In [ ]:
from fastapi import FastAPI, HTTPException, Query
from gpiozero import OutputDevice, Button
import asyncio
from time import time
import math

# Mapping GPIO pins to relay (valve) numbers and positions in mm
valve_mapping = {
    34: (OutputDevice(18), 2567),
    24: (OutputDevice(17), 2205),
    14: (OutputDevice(23), 2318),
    13: (OutputDevice(27), 1715),
    33: (OutputDevice(24), 1850),
    23: (OutputDevice(22), 1508),
    22: (OutputDevice(25), 1116),
    12: (OutputDevice(12), 1332),
    41: (OutputDevice(6), 1310),
    42: (OutputDevice(16), 1128),
    43: (OutputDevice(13), 1524),
    44: (OutputDevice(20), 2135),
    31: (OutputDevice(19), 915),
    21: (OutputDevice(21), 713),
    11: (OutputDevice(26), 905),
    32: (OutputDevice(5), 1480 )
}

# Setup encoder
encoder_a = Button(2, pull_up=True)  # Phase A connected to GPIO 2 (Black wire)
encoder_b = Button(3, pull_up=True)  # Phase B connected to GPIO 3 (White wire)
encoder_z = Button(4, pull_up=True)  # Phase Z connected to GPIO 4 (Orange wire, optional)

# Initialize encoder tracking variables
pulse_count = 0
last_time = time()
PPR = 2000  # Pulses Per Revolution (this should match your encoder's specification)
wheel_diameter = 45.0  # Wheel diameter in cm
circumference = math.pi * wheel_diameter / 100  # Circumference in meters

# Callback function to count pulses
def count_pulse():
    global pulse_count
    pulse_count += 1

encoder_a.when_pressed = count_pulse

# Function to calculate the speed in m/s based on encoder pulses
def calculate_speed():
    global pulse_count, last_time
    current_time = time()
    elapsed_time = current_time - last_time
    last_time = current_time

    # Calculate speed (m/s)
    if elapsed_time > 0:
        revolutions = pulse_count / PPR
        speed_m_per_sec = (revolutions * circumference) / elapsed_time
    else:
        speed_m_per_sec = 0

    pulse_count = 0  # Reset pulse count after calculation
    return speed_m_per_sec

# Function to calculate the delay for a valve based on the distance to travel and speed
def calculate_delay(distance_to_travel, speed):
    if speed > 0:
        delay = distance_to_travel / (speed * 1000 *0.16666666666)  # delay in seconds based on speed in m/s
        return delay
    return None

# Asynchronous function to control relay based on the received valve number and encoder speed
async def control_valve(valve_number: int):
    if valve_number in valve_mapping:
        # Calculate speed and determine delay based on it
        speed = calculate_speed()
        delay = calculate_delay(valve_mapping[valve_number][1], speed)
        if delay is not None:
            print(f"Relay for valve {valve_number} will turn OFF after {delay:.2f} seconds.")
            await asyncio.sleep(delay)  # Wait until the object has traveled the specified distance

            valve_mapping[valve_number][0].off()
            print(f"Relay for valve {valve_number} is OFF for 1 second.")
            await asyncio.sleep(.25)  # Valve off for 1 second
            valve_mapping[valve_number][0].on()
            print(f"Relay for valve {valve_number} turned back ON.")
        else:
            print(f"Speed is 0, unable to calculate delay for valve {valve_number}.")
    else:
        print(f"Valve number {valve_number} not found.")

# Define the FastAPI app
app = FastAPI()

@app.get("/control-valve/")
async def control_valve_endpoint(valve_id: int = Query(..., description="The valve number to control")):
    asyncio.create_task(control_valve(valve_id))  # Spawn a new task for each command
    return {"status": "command received", "valve_number": valve_id}

@app.on_event("startup")
async def startup_event():
    # Initialize the GPIO pins (turn on all relays)
    for valve_number, (pin_number, target_position) in valve_mapping.items():
        pin_number.on()
    print("All relays are reset")

@app.on_event("shutdown")
async def shutdown_event():
    # If the server is shut down, ensure all relays are turned on
    for valve_number, (pin_number, target_position) in valve_mapping.items():
        pin_number.on()
    print("All relays are On. Server is off now...")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)